In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
def load_preproccess_scale_transform(csv_filepath):
    COLUMN_ORDER = [
        "SM_4",
        "Time",
        "ST_4",
        "temp",
        "humidity",
        "precip",
        "windspeed",
        "cloudcover",
        "conditions",
    ]

    dataset = pd.read_csv(csv_filepath)

    dataset.drop(
        [
            "Date",
            "SM_2",
            "SM_8",
            "SM_20",
            "SM_40",
            "ST_2",
            "ST_8",
            "ST_20",
            "ST_40",
            "solarenergy",
            "precipprob",
            "preciptype",
            "snow",
            "snowdepth",
            "windgust",
            "winddir",
            "sealevelpressure",
            "visibility",
            "solarenergy",
            "uvindex",
            "severerisk",
            "icon",
            "stations",
            "dew",
            "solarradiation",
            "Time",
            # "cloudcover",
            "feelslike",
            # "windspeed",
        ],
        inplace=True,
        axis=1,
    )

    dataset.dropna(inplace=True)
    # dataset = dataset[COLUMN_ORDER]
    # dataset["Time"] = dataset["Time"].apply(lambda x: int(x[:2]))

    X = dataset.iloc[:, 1:].values
    # print(X[0])

    y = dataset.iloc[:, 0].values

    ct = ColumnTransformer(
        transformers=[("encoder", OneHotEncoder(), [-1])], remainder="passthrough"
    )

    X = np.array(ct.fit_transform(X))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42
    )

    sc = MinMaxScaler()

    X_train[:, 9:] = sc.fit_transform(X_train[:, 9:])

    X_test[:, 9:] = sc.transform(X_test[:, 9:])

    X_train, y_train, X_test, y_test = map(
        np.asarray, [X_train, y_train, X_test, y_test]
    )

    X_train, X_test = map(lambda x: x.astype("float32"), [X_train, X_test])

    return (X_train, y_train), (X_test, y_test)

In [5]:
(X_train, y_train), (X_test, y_test) = load_preproccess_scale_transform(
    "C:/Users/ganes/OneDrive/Documents/GitHub/Project_LISA/water irrigation/datasets/prototype_final_dataset.csv")

In [ ]:
""" Model """
ANN_model = keras.Sequential(
    [
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.Dense(16, activation="relu"),
        keras.layers.Dense(8, activation="relu"),
        keras.layers.Dense(4, activation="relu"),
        keras.layers.Dense(2, activation="relu"),
        keras.layers.Dense(1),
    ]
)

ANN_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.mean_squared_error,
    metrics=keras.metrics.mean_absolute_error,
)
early_stopping = EarlyStopping(patience=5)
history = ANN_model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
)

ANN_model.summary()

plt.plot(history.history["loss"])
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
ann_ypred = ANN_model.predict(X_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Increase maximum depth
dtr_model = DecisionTreeRegressor(
    max_depth=10, random_state=0, min_samples_split=20)

# Fine-tune hyperparameters
dtr_model.fit(X_train, y_train)

dtr_ypred = dtr_model.predict(X_test).reshape(-1, 1)

dtr_mae = mean_absolute_error(y_test, dtr_ypred)
dtr_mse = mean_squared_error(y_test, dtr_ypred)
print("DTR MAE:", dtr_mae)
print("DTR MSE:", dtr_mse)

In [ ]:
from sklearn.svm import SVR

# Try different kernels
svr_models = [
    SVR(kernel="rbf", C=10, gamma=0.1),
    SVR(kernel="linear", C=10),
    SVR(kernel="poly", C=10, degree=3),
]

# Fine-tune hyperparameters
for model in svr_models:
    model.fit(X_train, y_train)

    svr_ypred = model.predict(X_test)

    svr_mae = mean_absolute_error(y_test, svr_ypred)
    svr_mse = mean_squared_error(y_test, svr_ypred)

    print(f"SVR {model.kernel} MAE: {svr_mae}, MSE: {svr_mse}")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Feature selection using L1 regularization
mlr_model = LinearRegression()
mlr_model.fit(X_train, y_train)

mlr_ypred = mlr_model.predict(X_test)

mlr_mae = mean_absolute_error(y_test, mlr_ypred)
mlr_mse = mean_squared_error(y_test, mlr_ypred)
print("MLR MAE:", mlr_mae)
print("MLR MSE:", mlr_mse)

# Consider adding polynomial features

poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

mlr_model_poly = LinearRegression()
mlr_model_poly.fit(X_train_poly, y_train)

mlr_poly_ypred = mlr_model_poly.predict(X_test_poly)

mlr_poly_mae = mean_absolute_error(y_test, mlr_poly_ypred)
mlr_poly_mse = mean_squared_error(y_test, mlr_poly_ypred)
print("MLR with Poly Features MAE:", mlr_poly_mae)
print("MLR with Poly Features MSE:", mlr_poly_mse)

In [ ]:
plt.figure(figsize=(15, 4))

plt.subplot(141)
plt.scatter(y_test[:150], y_test[:150], color="red")
plt.scatter(y_test[:150], ann_ypred[:150], color="green")
plt.title("ANN_model")

plt.subplot(142)
plt.scatter(y_test[:150], y_test[:150], color="red")
plt.scatter(y_test[:150], dtr_ypred[:150], color="green")
plt.title("dtr_model")

plt.subplot(143)
plt.scatter(y_test[:150], y_test[:150], color="red")
plt.scatter(y_test[:150], svr_ypred[:150], color="green")
plt.title("svr_models")

plt.subplot(144)
plt.scatter(y_test[:150], y_test[:150], color="red")
plt.scatter(y_test[:150], mlr_ypred[:150], color="green")
plt.title("mlr_model")

In [7]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Create a GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()

# Define the hyperparameter grid to search
param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.01, 0.1, 0.2],
    "max_depth": [3, 4, 5],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

# Use GridSearchCV to perform the grid search
grid_search = GridSearchCV(
    estimator=gb_regressor,
    param_grid=param_grid,
    scoring="neg_mean_squared_error",
    cv=5,
)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model on the test set using the best hyperparameters
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on Test Set:", mse)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Mean Squared Error on Test Set: 809.7291405159345
